In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
#import seaborn as sns
#sns.set(style='darkgrid', context='talk', palette='Dark2')

my_year_month_fmt = mdates.DateFormatter('%m/%y')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#DATA PREPARATION
data = np.load('/home/isabelle/Downloads/latest_coin_data.pkl')
#drop null data
data = data.dropna()


In [3]:
def share_cap_recurse(market_cap_list, num_coins, cap_ceiling_amount, final_cap_list):

    not_realloc_val = 0

    realloc_val = 0

    num_final = 0

    new_market_cap_list = []

    

    for i in range(0, len(market_cap_list)):

        if(market_cap_list[i] > cap_ceiling_amount):

            final_cap_list.append(cap_ceiling_amount)

            realloc_val += market_cap_list[i] - cap_ceiling_amount

            num_final += 1

        else:

            not_realloc_val += market_cap_list[i]

     

    if(num_final!=0):

        new_market_cap_list = [x * (1+(realloc_val/not_realloc_val)) for x in market_cap_list[num_final:]]

    else:

        new_market_cap_list = deepcopy(market_cap_list)

    

    if len(final_cap_list) == num_coins:

        return

    elif (new_market_cap_list[0] <= cap_ceiling_amount): #assumes ordered desc in market cap

        for cap in new_market_cap_list:

            final_cap_list.append(cap)

        return       

    else:

        share_cap_recurse(new_market_cap_list, num_coins, cap_ceiling_amount, final_cap_list)

        

def share_cap(market_cap_list, cap_ceiling_percent):  #CHANGE CAP CEILING HERE

    total_cap = sum(market_cap_list)

    cap_ceiling_amount = total_cap * cap_ceiling_percent

    final_cap_list = []

    

    share_cap_recurse(market_cap_list, len(market_cap_list), cap_ceiling_amount, final_cap_list)

    

    return (final_cap_list)

In [4]:
close_list = list()
# Making a list of all close_prices
for date in data.index:
    for column in data.columns:
        close_list.append(data[column][date]['close_price'])

# Splitting list into 133 lists (one per coin)
close_list2 = [close_list[i:i+916] for i  in range(0, len(close_list), 916)]

# Make an array
close_ar = np.asarray(close_list2)

# Make Close DataFrame
close_df = pd.DataFrame(data = close_ar, index = data.index, columns = data.columns)
close_df = close_df[close_df.columns[::-1]]

In [5]:
# Initialize a market_cap list
market_list = list()
# Making a list of all market_cap
for date in data.index:
    for column in data.columns:
        market_list.append(data[column][date]['market_cap'])

# Splitting list into 133 lists (one per coin)
market_list2 = [market_list[i:i+916] for i  in range(0, len(market_list), 916)]

# Make an array
market_ar = np.asarray(market_list2)
    
# Make Market Cap DataFrame
market_df = pd.DataFrame(data = market_ar, index = data.index, columns = data.columns)
market_df = market_df[market_df.columns[::-1]]

In [7]:
#Selecting data for startdate to enddate
selectedmarket = market_df.loc[:, 'Jan 01, 2017':'Jan 01, 2018':7]
selectedmarket = selectedmarket.iloc[:, :-1]
selectedclose = close_df.loc[:, 'Jan 01, 2017':'Jan 01, 2018':7]

#NEW CODE
dailyclose = close_df.loc[:,'Jan 01, 2017':'Jan 01, 2018']

In [8]:
#Generating % change
dailyclose1 = dailyclose.copy().T
dailychange = dailyclose1.pct_change(1).T
dailychange.head()

,"Jan 01, 2017","Jan 02, 2017","Jan 03, 2017","Jan 04, 2017","Jan 05, 2017","Jan 06, 2017","Jan 07, 2017","Jan 08, 2017","Jan 09, 2017","Jan 10, 2017",...,"Dec 23, 2017","Dec 24, 2017","Dec 25, 2017","Dec 26, 2017","Dec 27, 2017","Dec 28, 2017","Dec 29, 2017","Dec 30, 2017","Dec 31, 2017","Jan 01, 2018"
1337coin,NaN,1.000000,0.000000,-0.500000,0.000000,2.000000,-0.680000,-0.104167,-0.104651,4.194805,...,0.576000,-0.025381,-0.203125,0.843137,0.432624,-0.116337,-0.039216,-0.244898,0.308880,0.053097
advanced-internet-blocks,NaN,0.028986,0.262831,0.306238,-0.571491,-0.037150,0.271484,-0.116414,-0.009991,-0.132766,...,0.060448,-0.076601,-0.408948,1.161089,-0.296486,-0.075737,-0.212717,-0.079397,0.600480,-0.224440
adzcoin,NaN,-0.169716,0.202623,0.223904,0.016124,-0.078864,0.049543,-0.010490,-0.027937,0.043431,...,0.058605,0.064101,0.009190,0.133635,-0.107290,-0.059846,0.063524,-0.139308,0.306847,0.141225
aeon,NaN,0.225457,0.196427,0.173988,-0.204247,0.180750,-0.184778,-0.013758,-0.020267,-0.031057,...,0.145015,-0.018470,0.024194,-0.020997,0.050938,0.015306,0.592965,-0.132492,0.052727,-0.115717
anarchistsprime,NaN,0.078000,-0.021336,0.027488,-0.069188,-0.139742,0.019585,0.047458,-0.033441,0.025670,...,0.057711,-0.008508,-0.083188,0.329541,-0.225815,-0.033921,-0.008003,-0.187319,0.399609,-0.170480


In [9]:
cols = dailychange.columns[:-1]
dailychange = dailychange.drop('Jan 01, 2017', 1)
dailychange.columns = cols

In [10]:
#FOR LOOP FOR C10
topmc10 = {}
sc10 = {}
tc10 = {}
ps10 = {}
invest10 = {}
returns10 = {}
lg_returns10 = {}
selectedmc10 = {}
topcoins10 = {}

daily_change = {}
weighted_daily_change = {}
daily_returns = {}
final_daily_returns = list()

#for every date
for column in selectedmarket:
    #sort in descending order
    selectedmc10[column] = selectedmarket[column].sort_values(ascending=False)
    
    #take the top coins
    topmc10[column] = selectedmc10[column].head(10)
    #take the index of the top coins
    topcoins10[column] = topmc10[column].index
    
    #get the share cap of the top 20
    sc10[column] = share_cap(topmc10[column], 0.15)      #CHOICE OF CAP CEILING
    
    #total cap of top 20
    tc10[column] = sum(sc10[column])
    
    #percentage of total fund allocated to tether/C10- NEW CODE!!
    tetherfund = 0                    #INSERT CODE TO DETERMINE % OF FUNDS ALLOCATED TO TETHERFUND
    c10fund = 1 - tetherfund
    
    #percentage share to be allocated for top 20 - NEW CODE!!!
    ps10[column] = [x * c10fund / tc10[column] for x in sc10[column] ]
    
    #building dataframe of top20 coins by mc and %share to be invested
    invest10[column] = pd.DataFrame()
    invest10[column] = pd.DataFrame({'coin': topcoins10[column], 'pct_share': ps10[column]})
    invest10[column] = invest10[column].set_index('coin')
        
jlist = list(range(0,365,7))
    
for (j, column) in zip(jlist, selectedmarket):
    daily_change[column] = pd.DataFrame()
    daily_change[column] = pd.DataFrame(data = dailychange.iloc[:,j:j+7])
    daily_change[column] = daily_change[column].loc[topcoins10[column]]
    weighted_daily_change[column] = daily_change[column].mul(ps10[column], axis=0)
    
    daily_returns[column] = weighted_daily_change[column].sum()
    daily_returns[column].columns = ['daily_returns']
        
    final_daily_returns.extend(daily_returns[column])
    
    
    #for i, df in invest10.items():
     #   portfolio_simple_returns10[i] = (invest10[i].transpose().dot(returns10[i])) + tetherreturns[i]
      #  portfolio_total_returns10.append(portfolio_simple_returns10[i]['returns'])   

In [15]:
C10_daily_returns_15 = pd.DataFrame(data = final_daily_returns)
C10_daily_returns_15.columns = ['daily_returns']
C10_daily_returns_15['date'] = dailychange.T.iloc[:-1].index
C10_daily_returns_15 = C10_daily_returns_15.set_index('date')
C10_daily_returns_15.index = pd.to_datetime(C10_daily_returns_15.index)

In [17]:
import math

In [36]:
final_cum10 = (1 + C10_daily_returns_15).cumprod()

In [38]:
final_cum10.index = pd.to_datetime(final_cum10.index)
final_C10 = C10_daily_returns_15.merge(final_cum10, how='outer', left_index=True, right_index=True)
final_C10.columns = ['daily_simple_returns', 'cumulative_returns']
ts = pd.to_datetime("2017-12-30", format="%Y-%m-%d")
new_row = pd.DataFrame([[None, 1]], columns = ['daily_simple_returns', 'cumulative_returns'], index=[ts])
final_C10 = pd.concat([final_C10, pd.DataFrame(new_row)], ignore_index=False)
final_C10.daily_simple_returns = final_C10.daily_simple_returns.shift(1)
final_C10.cumulative_returns = final_C10.cumulative_returns.shift(1)
final_C10.at['2017-01-01', 'cumulative_returns'] = 1

In [23]:
def max_drawdown(X):
    mdd = 0
    peak = X[0]
    for x in X:
        if x > peak: 
            peak = x
        dd = (peak - x) / peak
        if dd > mdd:
            mdd = dd
    return mdd    

In [25]:
C10_daily_returns_15.head()

,daily_returns
date,
2017-01-01,0.035898
2017-01-02,0.044019
2017-01-03,0.115758
2017-01-04,-0.098277
2017-01-05,-0.077012


In [40]:
max_drawdown(final_C10['cumulative_returns'])

0.43429067940991034

In [41]:
def max_drawup(X):
    mdd = 0
    trough = X[0]
    for x in X:
        if x < trough: 
            trough = x
        dd = (x - trough) / trough
        if dd > mdd:
            mdd = dd
    return mdd    

In [42]:
max_drawup(final_C10['cumulative_returns'])

113.67965867946064

https://stackoverflow.com/questions/22607324/start-end-and-duration-of-maximum-drawdown-in-python